In [2]:
from functools import partial
from geopy.geocoders import Nominatim
import WazeRouteCalculator
geolocator = Nominatim(user_agent="useer_agent", domain='nominatim.openstreetmap.org')
geocode = partial(geolocator.geocode, language="it", country_codes = "ita")


In [3]:
from pandas_ods_reader import read_ods
import pandas as pd
path = "IndirizziBiblio_21-10-2019.ods"
df = read_ods(path, 'Questionari_normalizza')
df=df[df['Calendarizzata (sì/no)']=='sì']
df=df.astype({'Cap':'int32'})
df.head()

,Denominazione,Comune,Provincia,Indirizzo,Cap,Calendarizzata (sì/no)
0,Agnadello - Biblioteca Comunale,Agnadello,CR,"via Marconi, 6",26020,sì
1,Annicco - Biblioteca Comunale Remo Contardi,Annicco,CR,"piazza Giuseppe Garibaldi, 6",26021,sì
3,Bagnolo Cremasco - Biblioteca Comunale,Bagnolo Cremasco,CR,"piazza Roma, 3",26010,sì
4,Bonemerse - Biblioteca Comunale,Bonemerse,CR,"via Roma, 25",26040,sì
7,Camisano - Biblioteca Comunale,Camisano,CR,"via Trieste, 57",26010,sì


In [4]:
def getLocation(name):
    geolocator = Nominatim(user_agent="useer_agent")
    geocode = partial(geolocator.geocode, language="it")
    location =geolocator.geocode(name)
    return location.latitude, location.longitude
lat, lon = getLocation("Agnadello, marconi, 6")
#print(lat, lon)

In [5]:
def getName(row):
    name=row.Comune + ','
    try:
        name=name + row.Indirizzo[row.Indirizzo.lower().index('via')+3:len(row.Indirizzo)]
    except:
        try:
            name=name + row.Indirizzo[row.Indirizzo.lower().index('piazza')+6:len(row.Indirizzo)]
        except:
            try:
                name=name + row.Indirizzo[row.Indirizzo.lower().index('piazzetta')+9:len(row.Indirizzo)]
            except:
                name=name + row.Indirizzo[row.Indirizzo.lower().index('vicolo')+6:len(row.Indirizzo)]
           
    return name

In [6]:
from tqdm.notebook import tqdm as tqdm

In [23]:
#DO NOT RUN
import csv
x=[]
y=[""]
for i in range(len(df)):
    x.append(getName(df.iloc[i]))
x.append("Cremona, Sesto 39")

for i in range(len(df)):
    y.append(getName(df.iloc[i]))
y.append("Ostiano")
y.append("San Giovanni")

with open('times.csv', mode='a') as time_file:
        time_file = csv.writer(time_file, delimiter=';')
        time_file.writerow(y)

with open('distances.csv', mode='a') as distance_file:
    distance_file = csv.writer(distance_file, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    distance_file.writerow(y)

y.pop(0)


In [44]:
import time
dataf=pd.read_csv("times.csv", encoding = "ISO-8859-1", delimiter=";", index_col=0, error_bad_lines=False)
start=len(dataf)
while start<15:
    try:
        time.sleep(5)
        dataf=pd.read_csv("times.csv", encoding = "ISO-8859-1", delimiter=";", index_col=0, error_bad_lines=False)
        start=len(dataf)
        for j in range(start, len(x)):
            lat, lon=getLocation(x[j])
            from_address=str(lat)+", "+str(lon)
            double_error = False
            t=[x[j]]
            d=[x[j]]
            for i in tqdm(range(len(y)), desc=str(j)):
                region = 'EU'
                lat, lon = getLocation(y[i])
                to_address=str(lat)+", "+str(lon)
                try:
                    route = WazeRouteCalculator.WazeRouteCalculator(from_address, to_address, region)
                    route_time, route_distance = route.calc_route_info(real_time=False)
                    double_error=False
                except:
                    i=i-1
                    if double_error==True:
                        continue
                    double_error=True
                t.append(route_time)
                d.append(route_distance)
            with open('times.csv', mode='a') as time_file:
                time_file = csv.writer(time_file, delimiter=';', quoting=csv.QUOTE_MINIMAL)
                time_file.writerow(t)

            with open('distances.csv', mode='a') as distance_file:
                distance_file = csv.writer(distance_file, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                distance_file.writerow(d)
    except:
        pass

In [38]:
dataf=pd.read_csv("times.csv", encoding = "ISO-8859-1", delimiter=";", index_col=0, error_bad_lines=False)
dataf

,"Agnadello, Marconi, 6","Annicco, Giuseppe Garibaldi, 6","Bagnolo Cremasco, Roma, 3","Bonemerse, Roma, 25","Camisano, Trieste, 57","Capergnanica, XI febbraio, 7","Capralba, Piave, 2","Casalbuttano ed Uniti, Jacini, 23","Casale Cremasco-Vidolasco, Roma, 56","Casalmaggiore, Guglielmo Marconi, 8",...,"Trescore Cremasco, Vittorio Veneto, 1","Trigolo, Roma, 26","Vaiano Cremasco, Gloriosi Caduti, 3","Vailate, Dante Alighieri, 7","Vescovato, Roma, 44","Cremona, Felice Cavallotti, 2","Cremona, Palosca n. 2","Cremona, Palosca n. 2.1",Ostiano,San Giovanni
"Agnadello, Marconi, 6",0.18333333333333332,41.266.666.666.666.600,17.033.333.333.333.300,5.853.333.333.333.330,22.883.333.333.333.300,17.000000,10.333.333.333.333.300,4.606.666.666.666.660,21.733.333.333.333.300,87.65,...,12.55,32.95,12.65,7.166.666.666.666.660,6.178.333.333.333.330,5.518.333.333.333.330,5.606.666.666.666.660,5.606.666.666.666.660,66.7,66.7
"Annicco, Giuseppe Garibaldi, 6",3.996.666.666.666.660,0.0,33.1,25.483.333.333.333.300,33.65,28.200000,38.266.666.666.666.600,10.55,3.261.666.666.666.660,5.471.666.666.666.660,...,3.571.666.666.666.660,15.266.666.666.666.600,32.85,43.05,28.85,22.333.333.333.333.300,23.05,23.05,37.766.666.666.666.600,37.766.666.666.666.600
"Bagnolo Cremasco, Roma, 3",15.15,32.266.666.666.666.600,0.0,49.483.333.333.333.300,22.9,7.550000,19.583.333.333.333.300,40.35,21.866.666.666.666.600,78.55,...,12.116.666.666.666.600,24.75,5.983.333.333.333.330,21.583.333.333.333.300,52.8,46.166.666.666.666.600,47.083.333.333.333.300,47.083.333.333.333.300,61.85,61.85
"Bonemerse, Roma, 25",57.733333333333334,24.983333333333334,50.85,0.0,51.016666666666666,46.166667,55.6,21.466666666666665,50.05,32.416666666666664,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Camisano, Trieste, 57",27.95,24.033333333333335,19.216666666666665,19.216666666666665,33.05,74.150000,63.1,14.45,16.5,74.56666666666666,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Capergnanica, XI febbraio, 7",17.716666666666665,28.6,9.216666666666667,45.7,19.733333333333334,0.000000,18.933333333333334,37.35,18.716666666666665,74.21666666666667,...,15.25,21.65,10.8,23.616666666666667,48.8,42.666666666666664,43.28333333333333,43.28333333333333,57.78333333333333,57.78333333333333
"Capralba, Piave, 2",9.866666666666667,36.916666666666664,19.95,55.53333333333333,15.75,18.083333,0.11666666666666667,37.65,13.0,13.0,...,9.033333333333333,26.966666666666665,16.683333333333334,6.9,58.7,52.583333333333336,53.2,53.2,64.45,64.45
"Casalbuttano ed Uniti, Jacini, 23",47.083333333333336,11.633333333333333,41.9,23.05,33.31666666666667,38.000000,38.4,0.36666666666666664,32.81666666666667,51.55,...,37.63333333333333,18.716666666666665,41.733333333333334,43.21666666666667,26.15,20.433333333333334,20.6,20.6,31.0,31.0
"Casale Cremasco-Vidolasco, Roma, 56",21.5,31.75,22.516666666666666,50.38333333333333,6.2,18.516667,12.816666666666666,32.38333333333333,0.0,78.95,...,12.033333333333333,21.783333333333335,19.65,17.133333333333333,53.583333333333336,47.416666666666664,47.95,47.95,59.21666666666667,59.21666666666667
"Casalmaggiore, Guglielmo Marconi, 8",86.86666666666666,54.13333333333333,79.63333333333334,33.38333333333333,80.4,74.866667,84.28333333333333,50.8,79.35,0.0,...,82.28333333333333,63.38333333333333,79.91666666666667,89.06666666666666,33.7,44.766666666666666,34.03333333333333,34.03333333333333,35.13333333333333,35.13333333333333


range(9, 68)
